# Healthcare Vendor Consolidation ML System

**Course:** AIM 5004-1 Predictive Modeling  
**Author:** Gregory  
**Date:** December 2025

---

## Problem Statement

**Stakeholder:** Private Equity (PE) Operating Partner managing 200-500 dental practice sites.

**Problem:** Fragmented vendor landscape across acquired dental practices. Need to identify optimal vendor consolidation opportunities.

**ML Task:** Link prediction in a bipartite graph (sites ↔ vendors) to predict which site-vendor relationships will be most successful.

**Value Proposition:**
- Projected annual savings: **$40,486**
- Risk-adjusted savings: **$38,075**
- Days-A/R improvement: **-1.5 days**
- Total value: **$61,596**

---

## 1. Setup and Imports

In [ ]:
import sys
from pathlib import Path

# Add parent directory for imports
sys.path.insert(0, str(Path('.').resolve().parent))

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Paths
BASE_DIR = Path('.').resolve().parent
DATA_DIR = BASE_DIR / 'data'
RESULTS_DIR = BASE_DIR / 'results'
MODELS_DIR = BASE_DIR / 'models'
BUSINESS_DIR = BASE_DIR / 'business'

print(f"Base directory: {BASE_DIR}")
print(f"Data files: {list(DATA_DIR.glob('*.csv'))}")

---

## 2. Data Overview

Synthetic dataset representing a PE-backed dental practice network.

In [ ]:
# Load core data
sites = pd.read_csv(DATA_DIR / 'sites.csv')
vendors = pd.read_csv(DATA_DIR / 'vendors.csv')
contracts = pd.read_csv(DATA_DIR / 'contracts_2019_2024.csv')
integration = pd.read_csv(DATA_DIR / 'integration_matrix.csv')

print("=" * 60)
print("DATA SUMMARY")
print("=" * 60)
print(f"\nSites: {len(sites)} dental practices")
print(f"Vendors: {len(vendors)} service providers")
print(f"Contracts: {len(contracts)} historical relationships")
print(f"Integration pairs: {len(integration)} site-vendor combinations")

In [ ]:
# Site distribution by region
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Region distribution
sites['region'].value_counts().plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Sites by Region')
axes[0].set_xlabel('Region')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)

# Vendor categories
vendors['category'].value_counts().plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Vendors by Category')
axes[1].set_xlabel('Category')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=45)

# Contracts over time
contracts['year'] = pd.to_datetime(contracts['start_date']).dt.year
contracts['year'].value_counts().sort_index().plot(kind='bar', ax=axes[2], color='seagreen')
axes[2].set_title('Contracts by Year')
axes[2].set_xlabel('Year')
axes[2].set_ylabel('New Contracts')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Integration quality distribution (KEY FEATURE)
print("\nIntegration Quality Distribution:")
print("=" * 40)
quality_map = {0: 'None', 1: 'Partial', 2: 'Full API'}
integration['quality_label'] = integration['integration_quality'].map(quality_map)
print(integration['quality_label'].value_counts())

fig, ax = plt.subplots(figsize=(6, 4))
colors = ['#ff6b6b', '#ffd93d', '#6bcb77']
integration['quality_label'].value_counts().plot(kind='pie', ax=ax, colors=colors, autopct='%1.1f%%')
ax.set_title('Integration Quality Distribution\n(Key Signal: 25.5% of model performance)')
ax.set_ylabel('')
plt.show()

---

## 3. Temporal Train/Validation Split

**Rubric Requirement:** Proper train/test/validation split

- **Training:** 2019-2022 contracts (historical)
- **Validation:** 2023-2024 contracts (future prediction)

In [ ]:
# Load temporal splits
train_edges = pd.read_csv(DATA_DIR / 'train_2019_2022.csv')
val_edges = pd.read_csv(DATA_DIR / 'dev_2023_2024.csv')

print("TEMPORAL SPLIT")
print("=" * 40)
print(f"Training edges (2019-2022): {len(train_edges)}")
print(f"Validation edges (2023-2024): {len(val_edges)}")
print(f"\nTask: Predict 2023-2024 contracts using 2019-2022 graph structure")

---

## 4. Model Tier Progression

**Rubric Requirement:** Baseline → Simple → Complex model progression

### Tier 1: Heuristic Baselines

Rule-based methods using graph topology only.

In [ ]:
# Tier 1 Results (pre-computed)
tier1_results = {
    'Jaccard Similarity': 0.164,
    'Peer Count': 0.171,
    'Rule-Based Composite': 0.113
}

print("TIER 1: HEURISTIC BASELINES")
print("=" * 40)
for method, pr_auc in tier1_results.items():
    print(f"  {method}: PR-AUC = {pr_auc:.3f}")

print("\nKey Finding: Heuristics perform poorly because they ignore")
print("             node features and edge attributes.")

### Tier 2: Gradient Boosting (LightGBM)

Traditional ML with handcrafted features.

In [ ]:
# Tier 2 Results (pre-computed)
tier2_results = {
    'PR-AUC': 0.937,
    'ROC-AUC': 0.924,
    'Features': 30
}

print("TIER 2: LIGHTGBM BASELINE")
print("=" * 40)
print(f"  PR-AUC: {tier2_results['PR-AUC']:.3f}")
print(f"  ROC-AUC: {tier2_results['ROC-AUC']:.3f}")
print(f"  Handcrafted features: {tier2_results['Features']}")

print("\nFeatures used:")
print("  - Site: region, EHR type, revenue tier")
print("  - Vendor: category, pricing tier")
print("  - Pair: integration_quality, historical counts")

print("\nKey Finding: Strong baseline - requires extensive feature engineering.")

### Tier 4: R-GCN (Primary Model)

**Relational Graph Convolutional Network** with edge-type-specific learning.

```
Architecture:
├── FastRGCNConv Layer 1 (3 relation types for integration_quality)
├── FastRGCNConv Layer 2
└── MLP Decoder with edge features
```

In [ ]:
# Load final metrics
with open(RESULTS_DIR / 'test_metrics.json') as f:
    metrics = json.load(f)

print("TIER 4: R-GCN (PRIMARY MODEL)")
print("=" * 50)
print(f"\nLink Prediction Task:")
print(f"  PR-AUC: {metrics['link_prediction']['metrics']['pr_auc']:.4f}")
print(f"  ROC-AUC: {metrics['link_prediction']['metrics']['roc_auc']:.4f}")
print(f"  Accuracy: {metrics['link_prediction']['metrics']['accuracy']:.0%}")

print(f"\nMulti-Task Outputs:")
print(f"  Risk Prediction MAE: {metrics['risk_head']['metrics']['mae']:.2f} days")
print(f"  Calibration ECE: {metrics['calibration']['metrics']['ece_after']:.4f}")
print(f"  Clustering Silhouette: {metrics['clustering']['metrics']['silhouette']:.4f}")

print(f"\nBest Hyperparameters:")
print(f"  Hidden channels: 128")
print(f"  Output channels: 64")
print(f"  Learning rate: 0.01")
print(f"  Edge dropout: 0.4")
print(f"  Best epoch: {metrics['metadata']['best_epoch']}")

---

## 5. Model Comparison

In [ ]:
# Create comparison DataFrame
comparison = pd.DataFrame([
    {'Model': 'Peer Count (Tier 1)', 'PR-AUC': 0.171, 'Type': 'Heuristic'},
    {'Model': 'LightGBM (Tier 2)', 'PR-AUC': 0.937, 'Type': 'Traditional ML'},
    {'Model': 'R-GCN (Tier 4)', 'PR-AUC': 0.9407, 'Type': 'Graph Neural Network'},
    {'Model': 'KumoRFM (External)', 'PR-AUC': 0.6209, 'Type': 'Foundation Model'},
])

# Sort by PR-AUC
comparison = comparison.sort_values('PR-AUC', ascending=True)

# Plot
fig, ax = plt.subplots(figsize=(10, 5))
colors = {'Heuristic': '#ff6b6b', 'Traditional ML': '#4ecdc4', 
          'Graph Neural Network': '#2ecc71', 'Foundation Model': '#9b59b6'}
bar_colors = [colors[t] for t in comparison['Type']]

bars = ax.barh(comparison['Model'], comparison['PR-AUC'], color=bar_colors)
ax.set_xlabel('PR-AUC', fontsize=12)
ax.set_title('Model Comparison: PR-AUC Performance', fontsize=14)
ax.set_xlim(0, 1.0)

# Add value labels
for bar, val in zip(bars, comparison['PR-AUC']):
    ax.text(val + 0.02, bar.get_y() + bar.get_height()/2, 
            f'{val:.4f}', va='center', fontsize=11)

# Add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=c, label=t) for t, c in colors.items()]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.show()

print("\nKey Finding: R-GCN beats GBM by +0.0037 PR-AUC (0.9407 vs 0.937)")
print("             R-GCN outperforms KumoRFM foundation model by +32%")

---

## 6. Ablation Studies

**Rubric Requirement:** Techniques for improving performance

In [ ]:
# Load ablation results
ablation = pd.read_csv(RESULTS_DIR / 'ablation_results.csv')

print("ABLATION STUDIES")
print("=" * 60)
print(f"\nBaseline PR-AUC: 0.9407\n")

# Show key ablations
key_ablations = ablation[ablation['experiment'].str.contains('integration|site_region|vendor_category')]
key_ablations = key_ablations[['experiment', 'ablated_pr_auc', 'delta']].copy()
key_ablations['impact'] = key_ablations['delta'].apply(
    lambda x: 'DOMINANT' if x > 0.2 else ('Major' if x > 0.01 else 'Minor')
)
print(key_ablations.to_string(index=False))

In [ ]:
# Ablation visualization
fig, ax = plt.subplots(figsize=(10, 5))

# Filter for granular feature ablations
feature_ablations = ablation[ablation['experiment'].str.contains('granular|integration')].copy()
feature_ablations['feature'] = feature_ablations['experiment'].str.replace('granular_no_', '').str.replace('_', ' ').str.title()
feature_ablations = feature_ablations.sort_values('delta', ascending=True)

colors = ['#e74c3c' if d > 0.1 else '#f39c12' if d > 0.01 else '#27ae60' 
          for d in feature_ablations['delta']]

bars = ax.barh(feature_ablations['feature'], feature_ablations['delta'], color=colors)
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax.set_xlabel('PR-AUC Drop When Removed', fontsize=12)
ax.set_title('Feature Importance via Ablation', fontsize=14)

# Add value labels
for bar, val in zip(bars, feature_ablations['delta']):
    ax.text(val + 0.005, bar.get_y() + bar.get_height()/2, 
            f'-{val:.1%}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\nKey Insight: Integration quality is the DOMINANT signal (25.5% impact)")
print("             R-GCN's edge-type-specific architecture is optimal for this.")

---

## 7. External Validation: KumoRFM

Tested against state-of-the-art foundation model from PyG founders (Stanford/Kumo.ai).

In [ ]:
# KumoRFM comparison
kumo_results = {
    'R-GCN (ours)': 0.9407,
    'KumoRFM': 0.6209
}

print("EXTERNAL VALIDATION: KumoRFM")
print("=" * 50)
print(f"\nKumoRFM: Graph foundation model from PyG founders")
print(f"         Pre-trained on diverse graph datasets\n")

for model, pr_auc in kumo_results.items():
    print(f"  {model}: PR-AUC = {pr_auc:.4f}")

delta = kumo_results['R-GCN (ours)'] - kumo_results['KumoRFM']
print(f"\nR-GCN advantage: +{delta:.4f} ({delta/kumo_results['KumoRFM']*100:.0f}% relative improvement)")

print("\nAnalysis:")
print("  - KumoRFM uses generic graph transformer architecture")
print("  - R-GCN explicitly models edge types (integration_quality)")
print("  - When edge-type signal dominates, task-specific R-GCN wins")

---

## 8. Business Output

Convert model predictions to actionable recommendations.

In [ ]:
# Load business outputs
plan = pd.read_csv(BUSINESS_DIR / 'plan_table.csv')

print("QUARTERLY CONSOLIDATION PLAN")
print("=" * 50)
print(f"\nTotal recommendations: {len(plan)}")

# Summary by quarter
if 'quarter' in plan.columns:
    quarter_summary = plan.groupby('quarter').agg({
        'site_id': 'count'
    }).rename(columns={'site_id': 'recommendations'})
    print("\nBy Quarter:")
    print(quarter_summary)

# Summary by risk
if 'risk_label' in plan.columns:
    risk_summary = plan['risk_label'].value_counts()
    print("\nBy Risk Level:")
    print(risk_summary)

In [ ]:
# Sample recommendation card
with open(BUSINESS_DIR / 'sample_card.txt') as f:
    sample_card = f.read()

print("SAMPLE RECOMMENDATION CARD")
print(sample_card)

In [ ]:
# Business value summary
print("PROJECTED BUSINESS VALUE")
print("=" * 50)
print("\nAnnual Impact:")
print("  - Direct savings: $40,486")
print("  - Risk-adjusted: $38,075")
print("  - Days-A/R improvement: -1.5 days")
print("  - Working capital benefit: $23,521")
print("  -" + "-" * 30)
print("  - TOTAL VALUE: $61,596")

print("\nImplementation:")
print("  - 50 vendor consolidation opportunities identified")
print("  - Risk-based staging across Q1-Q4")
print("  - Low-risk recommendations prioritized for Q1")

---

## 9. Summary and Conclusions

In [ ]:
print("="*70)
print("SUMMARY")
print("="*70)

print("""
MODEL PROGRESSION:
  Tier 1 (Heuristics)  → PR-AUC: 0.171  (baseline)
  Tier 2 (LightGBM)    → PR-AUC: 0.937  (+449% vs heuristics)
  Tier 4 (R-GCN)       → PR-AUC: 0.9407 (+0.4% vs GBM, BEST)

KEY FINDINGS:
  1. Integration quality contributes 25.5% of model performance
  2. R-GCN's edge-type architecture captures this signal optimally
  3. Task-specific R-GCN beats KumoRFM foundation model by 32%
  4. Multi-task learning adds risk prediction (MAE: 1.14 days)

BUSINESS IMPACT:
  - 50 vendor consolidation opportunities identified
  - $61,596 total annual value projected
  - Risk-based quarterly staging for implementation

RUBRIC ALIGNMENT:
  ✓ Problem/Stakeholder/Value proposition defined
  ✓ Baseline → Simple → Complex model progression
  ✓ Proper temporal train/validation split
  ✓ Feature engineering and ablation studies
  ✓ Hyperparameter tuning (random search, 12 runs)
  ✓ External validation (KumoRFM comparison)
  ✓ Business-ready output with actionable recommendations
""")

print("="*70)

---

## References

1. **R-GCN Paper:** Schlichtkrull et al. (2018). "Modeling Relational Data with Graph Convolutional Networks." arXiv:1703.06103

2. **PyTorch Geometric:** Fey & Lenssen (2019). "Fast Graph Representation Learning with PyTorch Geometric."

3. **KumoRFM:** Kumo.ai (2024). Relational Foundation Model for graph-structured data.

---

*Generated with Claude Code*